In [2]:
!pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.9.9-py3-none-any.whl.metadata (11 kB)
  Using cached array_record-0.8.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.1 kB)
  Using cached dm_tree-0.1.9-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.4 kB)
  Using cached etils-1.13.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached immutabledict-4.2.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached simple_parsing-0.1.7-py3-none-any.whl.metadata (7.3 kB)
  Using cached tensorflow_metadata-1.17.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
Using cached tensorflow_datasets-4.9.9-py3-none-any.whl (5.3 MB)
Using cached array_record-0.8.3-cp

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Choose the genres you want (names must match TFDS label names)
SELECTED_GENRES = ["classical", "jazz", "metal", "pop", "rock"]
N_PER_GENRE = 50

ds, info = tfds.load("gtzan", split="train", with_info=True, as_supervised=True)
label_names = info.features["label"].names  # list of 10 genre names

# Map selected genre names to label ids
selected_label_ids = tf.constant([label_names.index(g) for g in SELECTED_GENRES], dtype=tf.int64)

def keep_selected(audio, label):
    # label is scalar int
    return tf.reduce_any(tf.equal(label, selected_label_ids))

filtered = ds.filter(keep_selected)

# Build a balanced subset: exactly N_PER_GENRE examples per selected label
def balanced_take(ds_in, label_ids, n_per_label):
    per_label = []
    for lid in label_ids.numpy().tolist():
        ds_l = ds_in.filter(lambda a, y, lid=lid: tf.equal(y, tf.cast(lid, tf.int64))).take(n_per_label)
        per_label.append(ds_l)
    # Concatenate and shuffle
    out = per_label[0]
    for d in per_label[1:]:
        out = out.concatenate(d)
    return out.shuffle(len(SELECTED_GENRES) * n_per_label, reshuffle_each_iteration=False)

subset = balanced_take(filtered, selected_label_ids, N_PER_GENRE)

print("Subset size:", tf.data.experimental.cardinality(subset).numpy())


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: 